# Chemanim Video Rendering (GPU)
Upload your chemanim source and render videos with GPU acceleration.

## Step 1: Install Dependencies

In [ ]:
# System dependencies
!apt-get update -qq
!apt-get install -y -qq xvfb libgl1-mesa-dev libglu1-mesa-dev freeglut3-dev mesa-utils libosmesa6-dev libglfw3 libglfw3-dev

# Python packages
!pip install -q manimgl biopython biotite

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package freeglut3:amd64.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../00-freeglut3_2.8.1-6_amd64.deb ...
Unpacking freeglut3:amd64 (2.8.1-6) ...
Selecting previously unselected package libglx-dev:amd64.
Preparing to unpack .../01-libglx-dev_1.4.0-1_amd64.deb ...
Unpacking libglx-dev:amd64 (1.4.0-1) ...
Selecting previously unselected package libgl-dev:amd64.
Preparing to unpack .../02-libgl-dev_1.4.0-1_amd64.deb ...
Unpacking libgl-dev:amd64 (1.4.0-1) ...
Selecting previously unselected package libglvnd-core-dev:amd64.
Preparing to unpack .../03-libglvnd-core-dev_1.4.0-1_amd64.deb ...
Unpacking libglvnd-core-dev:amd64 (1.4.0-1) ...
Selecting previously unselected package libegl-dev:amd64.
Preparing to unpack .../04-li

In [ ]:
# Start virtual display
!Xvfb :99 -screen 0 1920x1080x24 &
import os
os.environ['DISPLAY'] = ':99'

## Step 2: Upload Chemanim Source
Upload the `src/chemanim` folder contents

In [ ]:
# Create directory structure
!mkdir -p chemanim

# Upload files
from google.colab import files
print("Upload these files from src/chemanim/:")
print("  - __init__.py")
print("  - bio.py")
print("  - chem_object_3d.py")
print("  - chem_object.py")
print("  - core.py")
print("  - data.py")
uploaded = files.upload()

# Move to chemanim folder
for filename in uploaded.keys():
    !mv "{filename}" chemanim/

In [ ]:
# Add to path
import sys
sys.path.insert(0, '/content')

# Test import
from chemanim.bio import Protein
print("✓ Chemanim imported successfully!")

## Step 3: Upload Demo Script

In [ ]:
# Upload your demo file (e.g., demo_protein_16_hemoglobin.py)
print("Upload your demo .py file:")
demo_files = files.upload()

## Step 4: Render Video

In [ ]:
# List uploaded demo files
demo_name = list(demo_files.keys())[0]
print(f"Demo file: {demo_name}")

# Extract scene class name (usually in the file)
with open(demo_name, 'r') as f:
    content = f.read()
    import re
    scenes = re.findall(r'class (\w+)\(Scene\)', content)
    print(f"Found scenes: {scenes}")

In [ ]:
# EDIT THIS: Set your scene name
SCENE_NAME = "HemoglobinShowcase"  # Change to your scene class name
DEMO_FILE = demo_name  # Or set manually: "demo_protein_16_hemoglobin.py"

# Render
!python -m manimlib {DEMO_FILE} {SCENE_NAME} -ow --video_dir ./output

## Step 5: Download Rendered Video

In [ ]:
import glob

# Find videos
videos = glob.glob('output/*.mp4') + glob.glob('*.mp4')
print(f"Rendered videos: {videos}")

# Download
for v in videos:
    print(f"Downloading {v}...")
    files.download(v)

---
## Quick Render (All-in-One)
If you have issues with the step-by-step, use this embedded demo:

In [ ]:
%%writefile quick_demo.py
from manimlib import *
import sys
sys.path.insert(0, '/content')
from chemanim.bio import Protein

class QuickProteinDemo(Scene):
    def construct(self):
        frame = self.camera.frame
        frame.set_euler_angles(theta=30 * DEGREES, phi=70 * DEGREES)
        frame.scale(3.0)
        
        title = Text("Protein Demo", font_size=40).to_edge(UP)
        title.fix_in_frame()
        self.add(title)
        
        # Fetch protein
        mol = Protein.from_pdb_id("1CRN", download_dir=".", include_hydrogens=False)
        mol.apply_render_style("cartoon")
        mol.scale(2.0)
        mol.move_to(ORIGIN)
        self.add(mol)
        
        # Color schemes demo
        for scheme in ["sse", "chain", "rainbow", "amino_acid"]:
            mol.set_color_scheme(scheme)
            self.play(Rotate(mol, angle=90*DEGREES, axis=UP), run_time=1.5)
            self.wait(0.3)

In [ ]:
# Render the quick demo
!python -m manimlib quick_demo.py QuickProteinDemo -ow

In [ ]:
# Download
files.download('QuickProteinDemo.mp4')